In [ ]:
from music21 import *
import numpy as np
import os
from collections import Counter
from sklearn.model_selection import train_test_split

In [ ]:
from keras.layers import Conv1D, Dropout, MaxPool1D, Dense, Embedding, GlobalMaxPool1D
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
import keras.backend as k

In [ ]:
!pip install tensorflow==2.12.0
!pip install keras==2.12.0

In [ ]:
def read_midi(file):
    print("Loading music file", file)

    notes = []
    notes_to_parse = None

    midi = converter.parse(file) #parsing midi files

    grp_ins = instrument.partitionByInstrument(midi) #partitioning based on different instrument

    for part in grp_ins.parts:
        if 'Piano' in str(part):
            notes_to_parse = part.recurse()

            # finding whether particular element is a note
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch)) #note
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder)) #chord
    return np.array(notes)

In [ ]:
path = '/content/schubert'
files = [i for i in os.listdir(path) if i.endswith(".mid")]
notes_array = np.array([read_midi(os.path.join(path, i)) for i in files])
# Use os.path.join to create the correct file path

In [ ]:
#converting 2d array to 1d array
notes_ = [element for note_ in notes_array for element in note_]
unique_notes = list(set(notes_))
print(len(unique_notes))

freq = dict(Counter(notes_))
frequent_notes = [note_ for note_, count in freq.items() if count>=50]
print(len(frequent_notes))

In [ ]:
#creating new array frequent music
new_music = []
for notes in notes_array:
    temp = []
    for note_ in notes:
        if note_ in frequent_notes:
            temp.append(note_)
    new_music.append(temp)

new_music = np.array(new_music)

In [ ]:
#preparing input and output sequences

no_of_timesteps = 32
x = []
y = []

for note_ in new_music:
    for i in range(0, len(note_) - no_of_timesteps, 1):

        input_ = note_[i:i + no_of_timesteps]
        output = note_[i + no_of_timesteps]

        x.append(input_)
        y.append(output)

x=np.array(x)
y=np.array(y)

In [ ]:
# assigning unique integer to every note
unique_x = list(set(x.ravel()))
x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

#preparing input sequences
x_seq=[]
for i in x:
    temp=[]
    for j in i:
        #assigning unique integer to every note
        temp.append(x_note_to_int[j])
    x_seq.append(temp)

x_seq = np.array(x_seq)

In [ ]:
#integer sequence for output
unique_y = list(set(y))
y_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_y))
y_seq=np.array([y_note_to_int[i] for i in y])

# 80% training and rest 20% for evaluation
x_tr, x_val, y_tr, y_val = train_test_split(x_seq,y_seq,test_size=0.2,random_state=0)

In [ ]:
from keras.layers import Conv1D, Dropout, MaxPool1D, Dense, Embedding, GlobalMaxPool1D
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
import keras.backend as k

k.clear_session()
model = Sequential()

#embedding layer
model.add(Embedding(len(unique_x), 100, input_length=32,trainable=True))

model.add(Conv1D(64,3, padding='causal',activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))

model.add(Conv1D(128,3,activation='relu',dilation_rate=2,padding='causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))

model.add(Conv1D(256,3,activation='relu',dilation_rate=4,padding='causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))

#model.add(Conv1D(256,5,activation='relu'))
model.add(GlobalMaxPool1D())

model.add(Dense(256, activation='relu'))
model.add(Dense(len(unique_y), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

model.summary()

In [ ]:
mc=ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True,verbose=1)

In [ ]:
history = model.fit(np.array(x_tr),np.array(y_tr),batch_size=128,epochs=50, validation_data=(np.array(x_val),np.array(y_val)),verbose=1, callbacks=[mc])

In [ ]:
from tensorflow import keras
model = keras.models.load_model('/content/best_model.h5')

In [ ]:
# predicted integer values
import random
ind = np.random.randint(0,len(x_val)-1)

random_music = x_val[ind]

predictions=[]
for i in range(45):

    random_music = random_music.reshape(1,no_of_timesteps)

    prob  = model.predict(random_music)[0]
    y_pred= np.argmax(prob,axis=0)
    predictions.append(y_pred)

    random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
    random_music = random_music[1:]

print(predictions)

In [ ]:
# predicted integer values
import random
ind = np.random.randint(0,len(x_val)-1)

random_music = x_val[ind]

predictions=[]
for i in range(300):

    random_music = random_music.reshape(1,no_of_timesteps)

    prob  = model.predict(random_music)[0]

    # Sample from the probability distribution instead of argmax
    y_pred = np.random.choice(len(prob), p=prob)

    predictions.append(y_pred)

    random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
    random_music = random_music[1:]

print(predictions)

In [ ]:
import random

# Extended "Happy Birthday" sequence (add more notes here)
happy_birthday_notes = ['C4', 'C4', 'D4', 'C4', 'F4', 'E4',
                        'C4', 'C4', 'D4', 'C4', 'G4', 'F4',
                        'C4', 'C4', 'C5', 'A4', 'F4', 'E4',
                        'D4', 'B4', 'B4', 'A4', 'F4', 'G4',
                        'F4']

# Convert notes to integers
happy_birthday_seq = [x_note_to_int[note] for note in happy_birthday_notes if note in x_note_to_int]

# Use the 'Happy Birthday' sequence as the starting point
random_music = np.array(happy_birthday_seq)

predictions = []
for i in range(45):  # Generate 45 more notes
    # Pad the input sequence if it's shorter than no_of_timesteps
    if len(random_music) < no_of_timesteps:
        random_music = np.pad(random_music, (no_of_timesteps - len(random_music), 0), 'constant')

    random_music = random_music.reshape(1, no_of_timesteps)


    prob = model.predict(random_music)[0]
    y_pred = np.random.choice(len(prob), p=prob)
    predictions.append(y_pred)

    # Shift the input sequence and add the prediction
    random_music = np.insert(random_music[0], len(random_music[0]), y_pred)
    random_music = random_music[1:]
    random_music = random_music[-no_of_timesteps:]  # Keep only the last 'no_of_timesteps' elements

print(predictions)

In [ ]:
# converting integer back into notes

x_int_to_note = dict((number, note_) for number, note_ in enumerate(unique_x))
predicted_notes = [x_int_to_note[i] for i in predictions]

In [ ]:
# converting the predictions to midi files
def convert_to_midi(prediction_output):
    offset = 0
    output_notes = []

    for pattern in prediction_output:

        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:

                cn=int(current_note)
                new_note = note.Note(cn)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)

            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)

        # pattern is a note
        else:

            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        offset += 1
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='music.mid')
    # Return midi_stream to make it accessible outside the function
    return midi_stream

# Call convert_to_midi and assign the result to midi_stream
midi_stream = convert_to_midi(predicted_notes)

# Now you can use midi_stream outside the function
midi_stream.write('midi', fp='/content/drive/My Drive/music.mid')
print("MIDI file saved to Google Drive.")

In [ ]:
import os
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq
from music21 import stream, note

# Set your Groq API Key
os.environ["GROQ_API_KEY"] = "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"  # 🔒 (consider using env vars securely)

# Function to generate MIDI notes
def generate_music_notes_langchain():
    chat = ChatGroq(model_name="llama3-8b-8192", temperature=0.7)

    prompt_template = ChatPromptTemplate.from_template(
        "You are a music AI that outputs a Python list of MIDI note integers (21 to 108). {input}"
    )

    llm_chain = LLMChain(llm=chat, prompt=prompt_template)

    prompt_input = {
        "input": "Generate a sequence of 32 random MIDI note integers between 21 and 108. for a happy birthday song"
    }
    generated = llm_chain.run(prompt_input)

    try:
        cleaned = generated.strip().replace("```python", "").replace("```", "").strip()
        if "[" in cleaned and "]" in cleaned:
            list_str = cleaned[cleaned.find("["):cleaned.find("]")+1]
            music_notes = eval(list_str)
        else:
            music_notes = [int(note.strip()) for note in cleaned.split(',')]
    except Exception as e:
        print("Error while parsing:", e)
        print("Generated content:", generated)
        music_notes = []

    return music_notes

# Function to convert list to MIDI
def convert_to_midi(note_int_list, duration=0.5):
    midi_stream = stream.Stream()
    for n in note_int_list:
        if 21 <= n <= 108:
            new_note = note.Note(n)
            new_note.quarterLength = duration
            midi_stream.append(new_note)
        else:
            print(f"Skipped invalid MIDI note: {n}")
    return midi_stream

# 🎵 Generate + Convert
predicted_notes = generate_music_notes_langchain()
print("Generated music notes (as integers):")
print(predicted_notes)

midi_stream = convert_to_midi(predicted_notes)

# 💾 Save MIDI
midi_stream.write('midi', fp='/content/music.mid')
print("✅ MIDI file saved to Google Drive.")


Generated music notes (as integers):
[69, 72, 65, 60, 64, 71, 74, 78, 83, 87, 91, 55, 59, 62, 65, 71, 73, 76, 80, 84, 88, 93, 97, 101, 72, 76, 80, 84, 89, 93, 97]
✅ MIDI file saved to Google Drive.


In [ ]:
from music21 import stream, note, chord

def advanced_happy_birthday_midi():
    s = stream.Stream()

    # Helper to add note
    def add_note(pitch, duration=0.5):
        n = note.Note(pitch)
        n.quarterLength = duration
        s.append(n)

    # Helper to add chord
    def add_chord(pitches, duration=0.5):
        c = chord.Chord(pitches)
        c.quarterLength = duration
        s.append(c)

    # Line 1: Happy Birthday to You
    add_note(64); add_note(64); add_note(66); add_note(64); add_note(69); add_note(68)

    # Chord: C major
    add_chord([60, 64, 67], duration=1)

    # Line 2: Happy Birthday to You
    add_note(64); add_note(64); add_note(66); add_note(64); add_note(71); add_note(69)

    # Chord: F major
    add_chord([65, 69, 72], duration=1)

    # Line 3: Happy Birthday dear [Name]
    add_note(64); add_note(64); add_note(76); add_note(73); add_note(69); add_note(68); add_note(66)

    # Chord: G major
    add_chord([67, 71, 74], duration=1)

    # Line 4: Happy Birthday to You
    add_note(74); add_note(74); add_note(73); add_note(69); add_note(71); add_note(69)

    # Chord: C major
    add_chord([60, 64, 67], duration=1)

    # Ending: Mini flourish
    add_note(72, 0.25); add_note(74, 0.25); add_note(76, 0.25); add_note(77, 0.5)

    # Final C major chord (octave stacked)
    add_chord([48, 60, 72], duration=2)  # Low C, Middle C, High C

    return s


In [ ]:
stream_result = advanced_happy_birthday_midi()
stream_result.write('midi', fp='/content/happy_birthday_advanced.mid')
print("🎉 Advanced Happy Birthday MIDI saved!")


🎉 Advanced Happy Birthday MIDI saved!
